In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta 

log = pd.read_csv('log.csv', header=None, names=['user_id', 'time', 'bet', 'win'])
us = pd.read_csv('users.csv', encoding='koi8_u', sep='\t')
us.columns = ['user_id', 'email', 'geo']

In [ ]:
log.info()
us.info()

In [ ]:
display(log.head())
display(us.head())

In [2]:
us.user_id = us.user_id.apply(lambda x: x.lower())

In [3]:
log = log[log.user_id != '#error']
log.user_id = log.user_id.str.split('- ').apply(lambda x: x[1])

In [4]:
log.bet = log['bet'].fillna(0)

def fillna_win(row):
    if np.isnan(row.win):
        row.win = 0
        return row.win
    return row.win


log['win'] = log.apply(lambda row: fillna_win(row), axis=1)

In [5]:
log['time'] = log.time.apply(lambda x: x[1:])
log['time'] = pd.to_datetime(log['time']) 

In [6]:
log['net'] = log.win - log.bet

In [7]:
display(log.head())
display(us.head())

,user_id,time,bet,win,net
0,user_919,2019-01-01 14:06:51,0.0,0.0,0.0
1,user_973,2019-01-01 14:51:16,0.0,0.0,0.0
2,user_903,2019-01-01 16:31:16,0.0,0.0,0.0
3,user_954,2019-01-01 17:17:51,0.0,0.0,0.0
4,user_954,2019-01-01 21:31:18,0.0,0.0,0.0


,user_id,email,geo
0,user_943,Accumanst@gmail.com,Ижевск
1,user_908,Advismowr@mail.ru,Ижевск
2,user_962,Anachso@ukr.net,Краснодар
3,user_973,Antecia@inbox.ru,Пермь
4,user_902,Balliaryva@ukr.net,NaN


In [8]:
df = pd.merge(log, us, on='user_id')  
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 974
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   user_id  975 non-null    object        
 1   time     975 non-null    datetime64[ns]
 2   bet      975 non-null    float64       
 3   win      975 non-null    float64       
 4   net      975 non-null    float64       
 5   email    967 non-null    object        
 6   geo      950 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 60.9+ KB


In [ ]:
user_bet_visit = df.groupby(['user_id'])[['bet', 'net']].sum()

In [ ]:
def visit_user(list_user_id):
    visit_list = []
    for user_id in list_user_id:
        visit_list.append(df.user_id.loc[df.user_id == user_id].count())
    return visit_list

list_user_id = user_bet_visit.index
user_bet_visit['visit'] = visit_user(list_user_id)

In [ ]:
def visit_user_zero(list_user_id):
    visit_zero = []
    for user_id in list_user_id:
        visit_zero.append(df.user_id.loc[(df.user_id == user_id)&(df.bet == 0)].count())
    return visit_zero

user_bet_visit['visit_zero'] = visit_user_zero(list_user_id)

In [ ]:
user_bet_visit['visit_zero'].loc[user_bet_visit.bet > 0].mean()

In [12]:
user_min_time = pd.DataFrame()
user_min_time['time1'] = df.loc[df.bet == 0].groupby(['user_id'])['time'].min()
user_min_time['time2'] = df.loc[df.bet > 0].groupby(['user_id'])['time'].min()
user_min_time['bet'] = df.groupby(['user_id'])['bet'].sum()

In [14]:
user_min_time = user_min_time[user_min_time.bet > 0]

In [15]:
(user_min_time.time2 - user_min_time.time1).mean()

Timedelta('46 days 06:54:48')

In [16]:
df.groupby(['geo'])['win'].sum().sort_values(ascending=False)

geo
Москва             11959741.0
Воронеж              184338.0
Санкт-Петербург      151007.0
Казань                97806.0
Ярославль             97441.0
Ижевск                84895.0
Красноярск            84767.0
Арзангелтск           74375.0
Пермь                 67734.0
Хабаровск             65459.0
Краснодар             62718.0
Ставрополь            46003.0
Екатеринбург          36682.0
Тюмень                 4701.0
Name: win, dtype: float64

In [19]:
df_geo = df.loc[df.bet > 0].groupby(['geo'])['bet'].mean()

In [20]:
df_geo.max() / df_geo.min()

127.81602335164835